In [1]:
from mlflow.tracking import MlflowClient
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

## Display all experiments in MLFlow database

In [3]:
client.search_experiments()

[<Experiment: artifact_location='file:///g:/Workspace/mlops-zoomcamp-2024/w02-experiment-tracking/scripts/playground/mlruns/1', creation_time=1718872759501, experiment_id='1', last_update_time=1718872759501, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='file:///g:/Workspace/mlops-zoomcamp-2024/w02-experiment-tracking/scripts/playground/mlruns/0', creation_time=1718872759442, experiment_id='0', last_update_time=1718872759442, lifecycle_stage='active', name='Default', tags={}>]

## Creating experiment

In [ ]:
client.create_experiment(name="new-experiment")

## Searching and displaying run informations 

In [15]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

runs

[<Run: data=<RunData: metrics={'rmse': 5.113606600306735}, params={'learning_rate': '0.08846978225899743',
  'max_depth': '39',
  'min_child_weight': '3.452827755032979',
  'objective': 'reg:linear',
  'reg_alpha': '0.01599788029454435',
  'reg_lambda': '0.23899980164449322',
  'seed': '42'}, tags={'mlflow.runName': 'caring-frog-509',
  'mlflow.source.name': 'g:\\Programs\\miniconda3\\envs\\experiment-tracking-env\\lib\\site-packages\\ipykernel_launcher.py',
  'mlflow.source.type': 'LOCAL',
  'mlflow.user': 'Fitri',
  'model': 'xgboost'}>, info=<RunInfo: artifact_uri='file:///g:/Workspace/mlops-zoomcamp-2024/w02-experiment-tracking/scripts/playground/mlruns/1/97a9eeee02854907abef2cdc5f9ec511/artifacts', end_time=1718876288730, experiment_id='1', lifecycle_stage='active', run_id='97a9eeee02854907abef2cdc5f9ec511', run_name='caring-frog-509', run_uuid='97a9eeee02854907abef2cdc5f9ec511', start_time=1718876076234, status='FINISHED', user_id='Fitri'>, inputs=<RunInputs: dataset_inputs=[]>>,

In [17]:
for run in runs:
    print(f"run id {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id 97a9eeee02854907abef2cdc5f9ec511, rmse: 5.1136
run id 818f595024684063872bc0bd1a53fc8d, rmse: 5.1212
run id deb2df26e5fa4e769b73ebf851458a35, rmse: 5.1219
run id 66af7fca1b334ed48b68b4caaee7ef42, rmse: 5.1232
run id e1468e1ea1e4491690d05371e595c968, rmse: 5.1259


## Registering Model

In [30]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

run_id = "274c0ad95d9f49d89f5d4e1611c2f1c5"
model_uri = f"runs/{run_id}/models_mlflow"
mlflow.register_model(model_uri=model_uri, name="nyc-xgboost")

In [21]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1718883614461, description='', last_updated_timestamp=1718883615381, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1718883615381, current_stage='None', description='', last_updated_timestamp=1718883615381, name='nyc-xgboost', run_id='274c0ad95d9f49d89f5d4e1611c2f1c5', run_link='', source='file:///g:/Workspace/mlops-zoomcamp-2024/w02-experiment-tracking/scripts/playground/mlruns/1/274c0ad95d9f49d89f5d4e1611c2f1c5/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=1>], name='nyc-xgboost', tags={}>]

In [22]:
model_name = "nyc-xgboost"
latest_version = client.get_latest_versions(name=model_name)

for version in latest_version:
    print(f"version: {version.version}")

version: 1


C:\Users\Fitri\AppData\Local\Temp\ipykernel_18780\3537920727.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest_version = client.get_latest_versions(name=model_name)


In [ ]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Staging",
    archive_existing_versions=False
)

In [ ]:
from datetime import datetime

model_version = 1
new_stage = "Staging"
date = datetime.today().date()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned into {new_stage} on {date}"
)

> `get_latest_version` and staging transitions `transition_model_version_stage` are deprecated since 2.9.0

## Load Model and Download Artifacts

In [36]:
from sklearn.metrics import root_mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [37]:
df = read_dataframe("../../data/green_tripdata_2023-03.parquet")

to download preprocessor artifacts:

In [40]:
import pickle

client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

X_test = preprocess(df,dv)
target = "duration"
y_test = df[target].values

test the model:

In [44]:
%time test_model(name=model_name, stage="None", X_test=X_test, y_test=y_test)

g:\Programs\miniconda3\envs\experiment-tracking-env\lib\site-packages\mlflow\store\artifact\utils\models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])
g:\Programs\miniconda3\envs\experiment-tracking-env\lib\site-packages\xgboost\core.py:160: UserWarning: [20:06:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: total: 21.1 s
Wall time: 1min 10s


{'rmse': 5.371714556554583}